In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
import os
import typing
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import Input, Model
# from tensorflow.keras.layers import Dense
from tqdm.notebook import tqdm
from statistics import mean,stdev
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
import random

from sklearn.metrics import *
from tcn import TCN
import wget
from getData import getData
from residual_momentum import resMomentum
import yfinance as yf
from random import random

from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
import concurrent.futures
import xgboost as xgb
import ta
from datetime import datetime, timedelta


In [16]:
#Low number of iterations for experimenting

In [17]:
stock_prices = pd.read_excel('data/halal_stocks.xlsx')
stock_prices = stock_prices.rename(columns={'Unnamed: 0':'Datetime'})
stock_prices['Datetime'] = pd.to_datetime(stock_prices['Datetime'])
stock_prices = stock_prices.set_index('Datetime')
stock_prices = stock_prices.dropna(axis=1,how='any')


In [18]:
lookbackperiod = 12
holdingperiod = 12
ranked_stocks,holding_returns,cumulative_returns = resMomentum(lookbackperiod,holdingperiod).getRankings(stock_prices)

[*********************100%%**********************]  1 of 1 completed


In [19]:
interval = '1d'
startdate = '2000-01-01'
stock_data = {}


In [20]:
for i in range(len(ranked_stocks)):
    stock_data[f'{ranked_stocks[i]}'] = yf.download(ranked_stocks[i],start=startdate,interval=interval)
    stock_data[f'{ranked_stocks[i]}'].index = pd.to_datetime(stock_data[f'{ranked_stocks[i]}'].index)
    
    stock_data[ranked_stocks[i]]['Tomorrow'] = stock_data[ranked_stocks[i]]["Close"].shift(-1)
    stock_data[ranked_stocks[i]]["Target"] = (stock_data[ranked_stocks[i]]["Tomorrow"] > stock_data[ranked_stocks[i]]["Close"]).astype(int)
    stock_data[ranked_stocks[i]] = stock_data[ranked_stocks[i]].iloc[:-1]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [21]:
end_date = datetime.today().strftime('%Y-%m-%d')
stock_dict = {}
for ticker_symbol in ranked_stocks:
    data = pd.DataFrame()
    
   



    data = yf.download(ticker_symbol, start="2000-01-01", end=end_date,interval='1d')

    
    data['SMA_5'] = data['Close'].rolling(window=5).mean()
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_20'] = data['Close'].rolling(window=20).mean()

    
    data['EMA_5'] = data['Close'].ewm(span=5, adjust=False).mean()
    data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()
    data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()

    
    data['RSI'] = ta.momentum.RSIIndicator(data['Close']).rsi()

    
    macd = ta.trend.MACD(data['Close'])
    data['MACD'] = macd.macd()
    data['MACD_Signal'] = macd.macd_signal()

    
    bollinger = ta.volatility.BollingerBands(data['Close'])
    data['Bollinger_High'] = bollinger.bollinger_hband()
    data['Bollinger_Low'] = bollinger.bollinger_lband()

    
    data['ATR'] = ta.volatility.AverageTrueRange(data['High'], data['Low'], data['Close']).average_true_range()

    
    stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'])
    data['Stoch_Oscillator'] = stoch.stoch()

    
    data['Williams_%R'] = ta.momentum.WilliamsRIndicator(data['High'], data['Low'], data['Close']).williams_r()

    
    data['MFI'] = ta.volume.MFIIndicator(data['High'], data['Low'], data['Close'], data['Volume']).money_flow_index()

    
    data['OBV'] = ta.volume.OnBalanceVolumeIndicator(data['Close'], data['Volume']).on_balance_volume()

    
    data['Accum_Dist_Line'] = ta.volume.AccDistIndexIndicator(data['High'], data['Low'], data['Close'], data['Volume']).acc_dist_index()

    
    data['Price_ROC'] = ta.momentum.ROCIndicator(data['Close']).roc()

    
    # market_data = yf.download('^GSPC', start="2020-01-01", end=end_date)
    # returns = data['Close'].pct_change().dropna()
    # market_returns = market_data['Close'].pct_change().dropna()
    # cov = np.cov(returns, market_returns)[0][1]
    # market_var = market_returns.var()
    # data['Beta'] = cov / market_var

    
    data['Hist_Volatility'] = data['Close'].pct_change().rolling(window=20).std() * np.sqrt(252)
    data["Tomorrow"] = data["Close"].shift(-1)
    data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
    data  = data.dropna()
    stock_dict[f'{ticker_symbol}'] = data


    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [22]:
df = stock_dict['ABM']

In [23]:
train = df.iloc[:-100]
test = df.iloc[-100:]
train, validation = train_test_split(train, test_size=0.2, shuffle=False)
import random


etas = [0.01,0.05,0.10,0.20,0.30,0.40,0.5,0.6,0.7]

params = {
    'max_depth': [5, 10, 15, 20,25],
    'subsample': [0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5,0.6,0.7,0.8,0.9,1.0],
    'gamma' : [0.00,0.05,0.10,0.15,0.20],
    'scale_pos_weight' : [30,40,50,300,400,500,600,700], 
}

iSol = {}
for key,values in params.items():
    iSol[key] = random.choice(values)

cSol = iSol
tStart = 100
tEnd = 1e-6
nLoops = 3
#overfit_threshold = 0.05 


frac = (tEnd/tStart)**(1.0/(nLoops-1.0))





# def predict(train, test, predictors, model):
#     model.fit(train[predictors], train["Target"])
#     preds = model.predict(test[predictors])
#     preds = pd.Series(preds, index=test.index, name="Predictions")
#     combined = pd.concat([test["Target"], preds], axis=1)
#     return combined

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)





def objective_function(cSol):
    
    model = XGBClassifier(**cSol)
    
    
    predictors = df.columns.drop(['Target','Tomorrow'])
    model.fit(train[predictors], train["Target"])
    
    
    predictions = backtest(df, model, predictors)
    score = precision_score(predictions["Target"], predictions["Predictions"])

    train_preds = model.predict(train[predictors])
    train_accuracy = precision_score(train["Target"], train_preds)

    
    valid_preds = model.predict(validation[predictors])
    valid_accuracy = precision_score(validation["Target"], valid_preds)


    overfit_measure = train_accuracy - valid_accuracy
    
    return score,overfit_measure


def simulated_annealing(T, frac, n):
    
    current_solution = {}

    

    for key,values in params.items():
        current_solution[key] = random.choice(values)
        current_solution.update({'eta':0.001})

    
    current_obj_val,overfit_measure = objective_function(current_solution)
    
    
    for i in range(n):
        
        new_solution = {}
        for  eta in etas:
            for key,values in params.items():
                new_solution[key] = random.choice(values)
                new_solution.update({'eta':eta})
            
            
            new_obj_val,overfit_measure= objective_function(new_solution)

            if (overfit_measure < 0.015) and(overfit_measure > -0.015):

                if new_obj_val>current_obj_val:
                    current_obj_val = new_obj_val
                    current_solution = new_solution
                    
                elif new_obj_val <= current_obj_val:
                    
                    p = np.exp(-(current_obj_val - new_obj_val) / T)
                    
                    if np.random.uniform(0, 1) < p:
                        current_solution = new_solution
                        current_obj_val = new_obj_val
                        
                
                
                T *= frac
                
                
                if T < tEnd:
                    break
                print('Current Overfit Measure : ',overfit_measure)
                print("Current Temp", T)
                print("Current Hyperparameters: ",current_solution)
                print("Current Objective Value: ",current_obj_val)
            
    

    return current_solution, current_obj_val,T,overfit_measure


best_solution, best_obj_val,T,overfit_measure = simulated_annealing(tStart, frac, nLoops)


print("Best Hyperparameters : ", best_solution)
print("Best Objective Function Value : ",best_obj_val)
print('Final Overfit Measure : ',overfit_measure)
print("Final Temp", T)




C:\Users\Maged\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Current Overfit Measure :  -0.009379640180945503
Current Temp 0.01
Current Hyperparameters:  {'max_depth': 20, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.0, 'scale_pos_weight': 400}
Current Objective Value:  0.510173880873104
Current Overfit Measure :  -0.01061301868692599
Current Temp 1.0000000000000002e-06
Current Hyperparameters:  {'max_depth': 20, 'eta': 0.05, 'subsample': 0.9, 'colsample_bytree': 1.0, 'gamma': 0.15, 'scale_pos_weight': 400}
Current Objective Value:  0.5097159940209267
Best Hyperparameters :  {'max_depth': 25, 'eta': 0.1, 'subsample': 0.5, 'colsample_bytree': 0.8, 'gamma': 0.1, 'scale_pos_weight': 30}
Best Objective Function Value :  0.5127367733507512
Final Overfit Measure :  -0.008942549456805282
Final Temp 1.0000000000000003e-18


In [24]:
from random import random
def fitness(X_train, y_train, X_test, y_test, position):
    selected_features = position > 0.5
    if not np.any(selected_features):
        return 1.0
    model = xgb.XGBClassifier(**best_solution)
    model.fit(X_train.iloc[:, selected_features], y_train)
    predictions = model.predict(X_test.iloc[:, selected_features])
    return 1.0 - precision_score(y_test, predictions)

def run_pso_on_stock_data(data, stock_name):
    X = data.drop(columns=['Tomorrow', 'Target'])
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    
    n_particles = 10
    n_features = X_train.shape[1]
    iterations = 2
    c1 = c2 = 1.5
    w = 0.5
    velocity_limit = 0.1

    
    particle_position = np.random.rand(n_particles, n_features) > 0.5
    particle_velocity = np.random.rand(n_particles, n_features) * velocity_limit
    personal_best_position = particle_position.copy()
    global_best_position = personal_best_position[np.random.choice(n_particles)]
    personal_best_fitness = np.array([fitness(X_train, y_train, X_test, y_test, pos) for pos in personal_best_position])
    global_best_fitness = personal_best_fitness.min()

    
    for iter_num in range(iterations):
        for i in range(n_particles):
            r1, r2 = random(), random()
            particle_velocity[i] = (w * particle_velocity[i] +
                                    c1 * r1 * (personal_best_position[i] != particle_position[i]) +
                                    c2 * r2 * (global_best_position != particle_position[i]))
            particle_velocity[i] = np.clip(particle_velocity[i], -velocity_limit, velocity_limit)
            flip_mask = np.random.rand(n_features) < particle_velocity[i]
            particle_position[i] = np.logical_xor(particle_position[i], flip_mask)

            current_fitness = fitness(X_train, y_train, X_test, y_test, particle_position[i])
            if current_fitness < personal_best_fitness[i]:
                personal_best_fitness[i] = current_fitness
                personal_best_position[i] = particle_position[i]
                if current_fitness < global_best_fitness:
                    global_best_fitness = current_fitness
                    global_best_position = particle_position[i]

        print(f"Stock: {stock_name}, Iteration: {iter_num + 1}/{iterations}, Current Best Fitness: {global_best_fitness}")

    return global_best_fitness, np.where(global_best_position > 0.5)[0]



best_features_dict = {}
for stock, data in stock_data.items():
    best_fitness, best_features_indices = run_pso_on_stock_data(data, stock)
    best_features = data.columns[best_features_indices]
    best_features_dict[stock] = best_features
    print(f"Stock: {stock}, Final Best Fitness: {best_fitness}, Best Features: {best_features}")

for stock, data in stock_data.items():
    if stock in best_features_dict:
        
        stock_data[stock] = data[best_features_dict[stock]]

Stock: FLWS, Iteration: 1/2, Current Best Fitness: 0.5130434782608695
Stock: FLWS, Iteration: 2/2, Current Best Fitness: 0.5130434782608695
Stock: FLWS, Final Best Fitness: 0.5130434782608695, Best Features: Index(['Low', 'Close'], dtype='object')
Stock: DDD, Iteration: 1/2, Current Best Fitness: 0.5163660654642619
Stock: DDD, Iteration: 2/2, Current Best Fitness: 0.5163660654642619
Stock: DDD, Final Best Fitness: 0.5163660654642619, Best Features: Index(['Low', 'Close', 'Adj Close'], dtype='object')
Stock: MMM, Iteration: 1/2, Current Best Fitness: 0.4803571428571428
Stock: MMM, Iteration: 2/2, Current Best Fitness: 0.4803571428571428
Stock: MMM, Final Best Fitness: 0.4803571428571428, Best Features: Index(['Open', 'Low', 'Close', 'Adj Close'], dtype='object')
Stock: AOS, Iteration: 1/2, Current Best Fitness: 0.47395833333333337
Stock: AOS, Iteration: 2/2, Current Best Fitness: 0.47395833333333337
Stock: AOS, Final Best Fitness: 0.47395833333333337, Best Features: Index(['Open', 'High